In [ ]:
# setup - don't forget to use GPU runtime setting
!git clone https://github.com/ml-jku/Chem-xLSTM.git
!apt-get install -y -qq openbabel
!pip -q install rdkit prolif dask scikit-learn
%cd Chem-xLSTM
!pip install -e .

Cloning into 'Chem-xLSTM'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 107 (delta 17), reused 61 (delta 11), pack-reused 36 (from 3)
Receiving objects: 100% (107/107), 83.74 MiB | 41.93 MiB/s, done.
Resolving deltas: 100% (21/21), done.


# Train an xLSTM model from scratch

In [17]:
# only for 3 epochs - but adjust to your liking
!python chemxlstm/train.py --model_class xLSTM --model_dim 256 --state_dim 64 --n_layers 4 --n_heads 1 \
--n_max_epochs 3 --batch_size 512 --device cuda:0 --learning_rate 5e-3 --dropout 0.25 --vocab_size 37 \
--sequence_length 100 --patience 5 --delta 1e-5 --save_per_epoch 3 --no_denovodesign \
--logdir ./models/ --training_molecules_path ./data/chemblv31/train.zip --val_molecules_path ./data/chemblv31/valid.zip

Current working directory:  /content/Chem-xLSTM
Total number of parameters:  1.644296 M
Total number of (trainable) parameters:  1.644296 M
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You chose "Don't visualize my results"
wandb: Tracking run with wandb version 0.21.4
wandb: W&B syncing is set to `offline` in this directory. Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
wandb: Run data is saved locally in /content/Chem-xLSTM/wandb/offline-run-20251001_124207-610ifwsc
Time taken for one forward pass:  0.2344803810119629 s
Input shape:  torch.Size([1, 100])
Output shape:  torch.Size([1, 100, 37])
Unsupported operator aten::embedding encountered 1 time(s)
Unsupported operator aten::add encountered 29 time(s)
Unsupported operator aten::silu encountered 8 time(s)
Unsupported operator aten::log_sigmoid encountered 4 time(s)
Unsupported operator aten::cumsum encountered 4

### Few-Shot Conditinoal Generation with a pretrained model


In [76]:
# download the pretrained models extract them and put them into models folder
!wget https://cloud.ml.jku.at/s/qpAS9iftYCN95by/download -O 'model.zip'
!unzip model.zip -d models
!unzip ./models/chem-xlstm-share/models.zip

--2025-10-01 15:50:19--  https://cloud.ml.jku.at/s/qpAS9iftYCN95by/download
Resolving cloud.ml.jku.at (cloud.ml.jku.at)... 140.78.90.41
Connecting to cloud.ml.jku.at (cloud.ml.jku.at)|140.78.90.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘model.zip’

model.zip               [        <=>         ] 105.54M  20.3MB/s    in 6.3s    

2025-10-01 15:50:27 (16.7 MB/s) - ‘model.zip’ saved [110669693]

Archive:  model.zip
   creating: models/chem-xlstm-share/
 extracting: models/chem-xlstm-share/models.zip  
Archive:  ./models/chem-xlstm-share/models.zip
   creating: models/chemblv31_pretrained/
   creating: models/chemblv31_pretrained/xLSTM-14.8M-ed512_hid64_l9_he8_267/
  inflating: models/chemblv31_pretrained/xLSTM-14.8M-ed512_hid64_l9_he8_267/init_arguments.json  
  inflating: models/chemblv31_pretrained/xLSTM-14.8M-ed512_hid64_l9_he8_267/model.pt  
   creating: models/icst_pretrained/
   creating: models/icst_pretrain

In [77]:
from chemxlstm import xLSTMforNTP, LSTMforNTP, MambaforNTP, GPTforNTP, xLSTMforNTP
model_path = '/content/Chem-xLSTM/models/icst_pretrained/xLSTM-15.0M-ed512_hid64_l9_he8_500'
device = 'cuda:0'
model = xLSTMforNTP.from_file(model_path, n_heads=8, device=device) #n_ssm has to be set

Loading model from /content/Chem-xLSTM/models/icst_pretrained/xLSTM-15.0M-ed512_hid64_l9_he8_500, using model class: xLSTMModel
Properties: {'mode': 'smiles', 'model_dim': 512, 'state_dim': 64, 'n_layers': 9, 'n_ssm': 1, 'dropout': 0.25, 'vocab_size': 200, 'sequence_length': 128, 'n_max_epochs': 100, 'learning_rate': 0.001, 'batch_size': 1, 'device': 'cuda:0', 'n_augmentations': 0, 'gpt_upj_factor': 4.0, 'kwargs': {}, 'warned_about_token': [], 'token2label': {'[PAD]': 0, '[BEG]': 1, 'C': 2, 'c': 3, '1': 4, '(': 5, '=': 6, 'O': 7, ')': 8, 'n': 9, '-': 10, '2': 11, '.': 12, 'Cl': 13, 'N': 14, '3': 15, 'S': 16, '/': 17, '[': 18, '+': 19, ']': 20, 'P': 21, '@': 22, 'H': 23, 'o': 24, 'F': 25, 'Br': 26, '#': 27, '4': 28, '5': 29, 'Na': 30, '6': 31, '7': 32, 's': 33, 'Si': 34, 'I': 35, 'K': 36, 'Au': 37, 'Pt': 38, 'Se': 39, 'Fe': 40, '8': 41, 'se': 42, 'Li': 43, '[END]': 44, 'B': 45, '9': 46, 'Cu': 47, 'Te': 48, 'As': 49, 'Ni': 50, 'Sb': 51, 'Ba': 52, 'Mn': 53, 'Ag': 54, 'Pb': 55, 'Eu': 56, '

In [79]:
# in the context put in the few samples form a distribution from which you want the designs to come from
seq, ll = model.design_molecules(n_designs=3, batch_size=1, temperature=1.4, context='CCCCCCCCCCCCCCC.CCCCCCCCCCCC.', debug=False, sequence_length=500)
seq

100%|██████████| 3/3 [00:22<00:00,  7.66s/it]


['CCCCCCCCCCCCCCC.CCCCCCCCCCCC.Clc1coo1)CNC2=CCN(c3ccco3)CC1(=O)%21NS(=O)(CO)(C(C)(OC(C)O)c(=O)n(CC)Cl)c1=O',
 'CCCCCCCCCCCCCCC.CCCCCCCCCCCC.C=[Pt+].C/C(C(Cl)F)OCCl)O[].CC#N-].Cl.[S-].C1=CC=C(C=C1)CC(=O)NIC(=O)NC2=C(C=CC(=C2)Br)ClNO)C=NC1=CCCN1C=NC2=CC=CN2)C.C(=O)(=O)C(=O)NC.Cl',
 'CCCCCCCCCCCCCCC.CCCCCCCCCCCC.CC1=CC(=C(C=C1)SC2=NC1=C(C=n1)C3CCcc(C3(C)CC45N(COC(C4C3=CC(C4CCCCC4)nc(N[[UNK][UNK]']